# classes

# general tester definition

In [1]:
# --- Función de modularización ---
def run_tests_for_step(step_number):
    """
    Ejecuta todos los tests para un paso específico.
    """
    print(f"--- Ejecutando todos los tests para el Paso {step_number} ---")
    
    total_tests = 0
    passed_tests = 0
    failed_tests = 0

    # Crea una copia estática de los elementos globales
    global_items = list(globals().items())
    
    for name, func in global_items:
        # Crea la cadena de búsqueda dinámica
        search_string = f"test_step{step_number}"
        if callable(func) and search_string in name:
            total_tests += 1
            try:
                print(f"\nEjecutando: {name}...")
                func()
                print(f"✅ {name} ha pasado.")
                passed_tests += 1
            except AssertionError as e:
                print(f"❌ {name} ha fallado. Error: {e}")
                failed_tests += 1
            except Exception as e:
                print(f"❌ {name} ha encontrado un error inesperado: {e}")
                failed_tests += 1

    print("\n--- Resumen de los Tests ---")
    print(f"Total de tests ejecutados: {total_tests}")
    print(f"Tests pasados: {passed_tests}")
    print(f"Tests fallados: {failed_tests}")
    print("----------------------------")


# class tester

# steps

## step 1

In [2]:
import numpy as np
import math

def step1_definitions(wavelength, layers, theta=0.0, phi=0.0):
    """
    Paso 1: Definiciones y Constantes
    - Define las variables de entrada.
    - Calcula el número de onda k0 y las componentes tangenciales kx, ky.
    """
    
    # 1.1 Definiciones iniciales
    # ----------------------------------------------------
    # Longitud de onda en el vacío (lambda) en metros
    # Espesor de la capa d en metros (se asume que está en la estructura 'layers')
    # Tensor de permitividad ε de cada capa (se asume en la estructura 'layers')
    # Ángulos de incidencia (theta) y azimutal (phi) en radianes
    
    # Se extrae el índice de refracción del medio de incidencia (primera capa) [cite: 12]
    # Se asume que 'layers' es una lista de diccionarios, y el primer diccionario
    # corresponde al medio de incidencia. Se debe incluir un campo 'n_in' o
    # 'refractive_index' en la estructura de datos.
    
    # Aquí un ejemplo de cómo podría ser la estructura de datos para layers
    # layers = [
    #   {'thickness': None, 'permittivity': np.array([[1,0,0],[0,1,0],[0,0,1]]), 'n_in': 1.0}, # Medio de incidencia
    #   {'thickness': 100e-9, 'permittivity': np.array([[...]]), ...},
    #   {'thickness': 200e-9, 'permittivity': np.array([[...]]), ...}
    # ]
    
    n_in = layers[0].get('n_in')
    if n_in is None:
        raise ValueError("El medio de incidencia (primera capa) debe tener un 'n_in' o 'refractive_index'.")
        
    # 1.2 Cálculo de constantes
    # ----------------------------------------------------
    # Número de onda en el vacío, k0 [cite: 7]
    k0 = (2 * math.pi) / wavelength 

    # Componentes tangenciales del vector de onda, kx y ky [cite: 8]
    # Estas componentes son invariantes en todas las capas del sistema [cite: 9]
    kx = k0 * n_in * math.sin(theta) * math.cos(phi) 
    ky = k0 * n_in * math.sin(theta) * math.sin(phi)
    
    # 1.3 Devolución de los resultados
    # ----------------------------------------------------
    return {
        'k0': k0,
        'kx': kx,
        'ky': ky
    }

In [3]:
import numpy as np
import cmath
import numpy as np
import math
import pytest

def test_step1():
    """
    Tests the step1_definitions function for correctness and numerical stability.
    """
    # Test Case 1: Standard inputs at normal incidence (theta=0)
    # --------------------------------------------------------------------------------------
    wavelength_1 = 600e-9
    theta_rad_1 = 0.0
    phi_rad_1 = 0.0
    
    # The first layer must have a refractive index for kx and ky calculations.
    layers_1 = [{'n_in': 1.0}]
    
    results_1 = step1_definitions(wavelength_1, layers_1, theta_rad_1, phi_rad_1)
    
    # Expected values
    expected_k0_1 = (2 * math.pi) / wavelength_1
    expected_kx_1 = 0.0 # sin(0) is 0
    expected_ky_1 = 0.0 # sin(0) is 0
    
    # Use pytest.approx for floating-point comparisons to handle small numerical differences
    assert results_1['k0'] == pytest.approx(expected_k0_1)
    assert results_1['kx'] == pytest.approx(expected_kx_1)
    assert results_1['ky'] == pytest.approx(expected_ky_1)

    # Test Case 2: Standard inputs with non-zero angles
    # --------------------------------------------------------------------------------------
    wavelength_2 = 450e-9
    theta_deg_2 = 45.0
    phi_deg_2 = 90.0 # Azimuthal angle along the y-axis
    theta_rad_2 = math.radians(theta_deg_2)
    phi_rad_2 = math.radians(phi_deg_2)

    layers_2 = [{'n_in': 1.5}] # A different refractive index
    
    results_2 = step1_definitions(wavelength_2, layers_2, theta_rad_2, phi_rad_2)
    
    # Expected values
    expected_k0_2 = (2 * math.pi) / wavelength_2
    # kx = k0 * n_in * sin(theta) * cos(phi)
    expected_kx_2 = expected_k0_2 * 1.5 * math.sin(theta_rad_2) * math.cos(phi_rad_2)
    # ky = k0 * n_in * sin(theta) * sin(phi)
    expected_ky_2 = expected_k0_2 * 1.5 * math.sin(theta_rad_2) * math.sin(phi_rad_2)

    assert results_2['k0'] == pytest.approx(expected_k0_2)
    assert results_2['kx'] == pytest.approx(expected_kx_2)
    assert results_2['ky'] == pytest.approx(expected_ky_2)

    # Test Case 3: Edge case with very small wavelength (numerical stability check)
    # --------------------------------------------------------------------------------------
    wavelength_3 = 1e-12 # A very small wavelength
    theta_rad_3 = math.pi / 4
    phi_rad_3 = math.pi / 3
    layers_3 = [{'n_in': 2.5}]
    
    results_3 = step1_definitions(wavelength_3, layers_3, theta_rad_3, phi_rad_3)
    
    # Expected values should be very large but finite
    expected_k0_3 = (2 * math.pi) / wavelength_3
    expected_kx_3 = expected_k0_3 * 2.5 * math.sin(theta_rad_3) * math.cos(phi_rad_3)
    expected_ky_3 = expected_k0_3 * 2.5 * math.sin(theta_rad_3) * math.sin(phi_rad_3)
    
    assert results_3['k0'] == pytest.approx(expected_k0_3)
    assert results_3['kx'] == pytest.approx(expected_kx_3)
    assert results_3['ky'] == pytest.approx(expected_ky_3)
    
    # Test Case 4: Invalid input (no refractive index)
    # --------------------------------------------------------------------------------------
    # The function should raise a ValueError if 'n_in' is missing from the first layer.
    layers_4 = [{'thickness': 100e-9}] # Missing 'n_in'
    with pytest.raises(ValueError):
        step1_definitions(600e-9, layers_4, 0, 0)

    print("All good")
# Ejecutar el test
if __name__ == "__main__":
    test_step1()

All good


## step 2

In [4]:
import numpy as np
from scipy.linalg import expm

import pytest
import numpy as np
import cmath
import math
from scipy.linalg import expm

def step2_build_matrix_A(epsilon_tensor, kx, ky, k0):
    """
    Constructs the Berreman 4x4 matrix A for wave propagation in an anisotropic medium.
    
    Parameters:
    - epsilon_tensor: 3x3 complex numpy array, the permittivity tensor of the medium.
    - kx, ky: Wave vector components in the x and y directions (in-plane).
    - k0: Free-space wave number (omega/c).
    
    Returns:
    - A: 4x4 complex numpy array, the Berreman system matrix multiplied by i*k0.
    """
    print("Using the CORRECTED version of step2_build_matrix_A.")
    epsilon_inv = np.linalg.inv(epsilon_tensor)
    e_xx_inv, e_xy_inv, e_xz_inv = epsilon_inv[0, 0], epsilon_inv[0, 1], epsilon_inv[0, 2]
    e_yx_inv, e_yy_inv, e_yz_inv = epsilon_inv[1, 0], epsilon_inv[1, 1], epsilon_inv[1, 2]
    e_zx_inv, e_zy_inv, e_zz_inv = epsilon_inv[2, 0], epsilon_inv[2, 1], epsilon_inv[2, 2]
    
    A = np.zeros((4, 4), dtype=complex)
    A[0, 0] = -kx * e_xz_inv / k0
    A[0, 1] = -ky * e_yz_inv / k0
    A[0, 2] = 0
    A[0, 3] = 1 - (kx**2 / k0**2) * (e_xx_inv - e_yy_inv)
    
    A[1, 0] = -ky * e_xz_inv / k0
    A[1, 1] = kx * e_yz_inv / k0
    A[1, 2] = -1 + (ky**2 / k0**2) * (e_xx_inv - e_yy_inv)
    A[1, 3] = 0
    
    A[2, 0] = ky * e_zz_inv / k0
    A[2, 1] = -kx * e_zz_inv / k0
    A[2, 2] = -(kx * ky / k0**2) * (e_xx_inv - e_yy_inv) + e_yy_inv
    A[2, 3] = -(kx * ky / k0**2) * (e_xx_inv - e_yy_inv) + e_yy_inv
    
    A[3, 0] = e_yz_inv
    A[3, 1] = -e_xz_inv
    A[3, 2] = -ky * e_xx_inv / k0
    A[3, 3] = kx * e_yy_inv / k0
    
    A = np.dot(1j * k0, A)
    return A

In [5]:
import pytest
import numpy as np


import pytest
import numpy as np
import math


def test_step2_isotropic_medium():
    print("\n--- Test 2.1: Isotropic Medium ---")
    wavelength = 800e-9
    layers = [{'n_in': 1.5}]
    theta = math.radians(30)
    phi = math.radians(0)
    
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    n = 1.5
    epsilon = np.diag([n**2, n**2, n**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)

    print(f"Matriz A:\n{A}")
    print(f"k0: {k0}")
    print(f"kx: {kx}")
    print(f"ky: {ky}")

    # Expected values based on standard Berreman matrix
    expected_A_01 = 0
    expected_A_02 = 0
    expected_A_03 = 1j * k0 * 1  # A[0, 3] = i k0 * 1
    expected_A_10 = 0
    expected_A_11 = 0  # A[1, 1] = i k0 * (kx * e_yz_inv / k0) = 0 since e_yz_inv = 0

    print(f"A[0, 1] actual: {A[0, 1]}, expected: {expected_A_01}")
    print(f"A[0, 2] actual: {A[0, 2]}, expected: {expected_A_02}")
    print(f"A[0, 3] actual: {A[0, 3]}, expected: {expected_A_03}")
    print(f"A[1, 0] actual: {A[1, 0]}, expected: {expected_A_10}")
    print(f"A[1, 1] actual: {A[1, 1]}, expected: {expected_A_11}")

    # Tolerance for floating-point comparisons
    assert np.allclose(A[0, 1], expected_A_01, atol=1e-8)
    assert np.allclose(A[0, 2], expected_A_02, atol=1e-8)
    assert np.allclose(A[0, 3], expected_A_03, atol=1e-8)
    assert np.allclose(A[1, 0], expected_A_10, atol=1e-8)
    assert np.allclose(A[1, 1], expected_A_11, atol=1e-8)
    
    print("✅ Test for isotropic medium passed.")

def test_step2_normal_incidence():
    print("\n--- Test 2.2: Normal Incidence ---")
    wavelength = 800e-9
    layers = [{'n_in': 1.5}]
    theta = math.radians(0)  # Normal incidence
    phi = math.radians(0)
    
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    epsilon = np.diag([1.5**2, 1.5**2, 1.5**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    
    # Expected matrix for normal incidence (kx = ky = 0)
    expected_A = 1j * k0 * np.array([
        [0, 0, 0, 1],
        [0, 0, -1, 0],
        [0, 0, 1/1.5**2, 1/1.5**2],
        [0, 0, 0, 0]
    ])

    print(f"Matriz A:\n{A}")
    print(f"Expected A:\n{expected_A}")
    
    assert np.allclose(A, expected_A, atol=1e-8)
    print("✅ Test for normal incidence passed.")

def test_step2_anisotropic_medium():
    print("\n--- Test 2.3: Uniaxial Anisotropic Medium ---")
    wavelength = 632.8e-9
    layers = [{'n_in': 1.0}]
    theta = math.radians(20)
    phi = math.radians(0)
    
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    # Uniaxial permittivity tensor
    e_o = 2.25  # Ordinary
    e_e = 2.6   # Extraordinary
    epsilon = np.diag([e_o, e_o, e_e])
    
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    
    # Validate key non-zero elements
    assert not np.allclose(A[0, 3], 0)  # A[0, 3] = i k0 * 1
    assert not np.allclose(A[1, 2], 0)  # A[1, 2] = i k0 * (-1)
    assert not np.any(np.isnan(A))
    assert not np.any(np.isinf(A))

    print(f"Matriz A:\n{A}")
    print("✅ Test for uniaxial anisotropic medium passed.")
    
def test_step2_T_matrix_numerical_stability():
    print("\n--- Test T-Matrix Estabilidad Numérica (Versión Corregida) ---")
    
    # Parámetros físicos típicos que antes causaban desbordamiento
    wavelength = 800e-9 # 800 nm
    d_large = 1e-6 # 1 micrómetro
    theta = math.radians(45)
    
    print(f"**Parámetros de entrada:**")
    print(f"Wavelength: {wavelength} m")
    print(f"Theta: {math.degrees(theta)} degrees")
    print(f"Large layer thickness (d): {d_large} m")
    
    layers = [{'n_in': 1.5}]
    constants = step1_definitions(wavelength, layers, theta)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    print(f"\n**Constantes calculadas (Paso 1):**")
    print(f"k0: {k0} 1/m")
    print(f"kx: {kx} 1/m")
    print(f"ky: {ky} 1/m")
    
    epsilon = np.diag([1.5**2, 1.5**2, 1.5**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    
    print(f"\n**Matriz de evolución A (Paso 2):**\n{A}")
    
    T = expm(A * d_large)
    
    print(f"\n**Matriz de propagación T (Paso 4):**\n{T}")
    
    assert not np.any(np.isnan(T))
    assert not np.any(np.isinf(T))
    
    print("✅ Test de estabilidad de la matriz T pasado.")
    
# --- Bloque de ejecución principal ---
if __name__ == "__main__":
    # Puedes cambiar este número para ejecutar los tests de otro paso
    run_tests_for_step(2)

--- Ejecutando todos los tests para el Paso 2 ---

Ejecutando: test_step2_isotropic_medium...

--- Test 2.1: Isotropic Medium ---
Using the CORRECTED version of step2_build_matrix_A.
Matriz A:
[[0.      +0.j         0.      +0.j         0.      +0.j
  0.+7853981.63397448j]
 [0.      +0.j         0.      +0.j         0.-7853981.63397448j
  0.      +0.j        ]
 [0.      +0.j         0.-2617993.87799149j 0.+3490658.50398866j
  0.+3490658.50398866j]
 [0.      +0.j         0.      +0.j         0.      +0.j
  0.+2617993.87799149j]]
k0: 7853981.633974483
kx: 5890486.225480861
ky: 0.0
A[0, 1] actual: 0j, expected: 0
A[0, 2] actual: 0j, expected: 0
A[0, 3] actual: 7853981.633974483j, expected: 7853981.633974483j
A[1, 0] actual: 0j, expected: 0
A[1, 1] actual: 0j, expected: 0
✅ Test for isotropic medium passed.
✅ test_step2_isotropic_medium ha pasado.

Ejecutando: test_step2_normal_incidence...

--- Test 2.2: Normal Incidence ---
Using the CORRECTED version of step2_build_matrix_A.
Matriz A:
[

## step 3

In [6]:

def step3_modes(A, epsilon, kx, ky, k0):
    """
    Dispatcher for Step 3: Mode Calculation.
    
    Analyzes eigenvalues of A to detect degeneracies or other exceptions.
    Reroutes to appropriate sub-function:
    - step3_general: For non-degenerate cases (distinct eigenvalues).
    - step3_degenerate: For degenerate cases (e.g., normal incidence in isotropic media).
    
    Parameters:
    - A: 4x4 complex numpy array, Berreman system matrix.
    - epsilon: 3x3 permittivity tensor (for degeneracy checks).
    - kx, ky, k0: Wave vector components (for analytical handling).
    
    Returns:
    - F_plus: 4x2 numpy array, forward-propagating modes.
    - F_minus: 4x2 numpy array, backward-propagating modes.
    """
    print("--- Step 3: Mode Calculation ---")
    # Input validation
    if A.shape != (4, 4):
        raise ValueError("Matrix A must be 4x4.")
    if epsilon.shape != (3, 3):
        raise ValueError("Epsilon must be 3x3.")
    
    # Compute eigenvalues
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"Eigenvalues: {eigenvalues}")
    
    # Check for degeneracies
    tolerance = 1e-6
    eigenvalue_diffs = np.abs(np.subtract.outer(eigenvalues, eigenvalues))
    is_degenerate = np.any((eigenvalue_diffs < tolerance) & (eigenvalue_diffs > 0))
    
    # Check for normal incidence and isotropic medium
    is_normal_incidence = (abs(kx) < 1e-10 and abs(ky) < 1e-10)
    is_isotropic = np.allclose(epsilon, np.diag([epsilon[0,0], epsilon[0,0], epsilon[0,0]]), atol=1e-6)
    
    if is_degenerate or (is_normal_incidence and is_isotropic):
        print("Degeneracy detected, routing to step3_degenerate.")
        return step3_degenerate(A, epsilon, kx, ky, k0)
    else:
        print("No degeneracy, routing to step3_general.")
        return step3_general(A, eigenvectors, eigenvalues)

def step3_general(A, eigenvectors, eigenvalues):
    """
    General case: Non-degenerate eigenvalues.
    Classify modes using Poynting vector S_z, with eigenvalue sorting as fallback.
    """
    print("--- step3_general: Processing non-degenerate case ---")
    tolerance = 1e-6
    modes = []
    
    # Compute S_z and collect mode data
    for i in range(4):
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        imag_eig = np.imag(eigenvalues[i])
        modes.append((v_j, Sz, imag_eig, i))
        print(f"Eigenvector {i+1}: {v_j}, S_z: {Sz}, Im(eigenvalue): {imag_eig}")
    
    # Sort modes by |S_z| descending, then by Im(eigenvalue) for tiebreaking
    modes.sort(key=lambda x: (abs(x[1]), x[2]), reverse=True)
    
    F_plus = []
    F_minus = []
    
    # Select top two forward and backward modes
    for v_j, Sz, imag_eig, i in modes:
        norm = np.sqrt(abs(Sz)) if abs(Sz) > tolerance else np.linalg.norm(v_j)
        if norm < tolerance:  # Avoid division by zero
            norm = 1.0
        v_norm = v_j / norm
        if Sz > tolerance and len(F_plus) < 2:
            F_plus.append(v_norm)
        elif Sz < -tolerance and len(F_minus) < 2:
            F_minus.append(v_norm)
        elif abs(Sz) <= tolerance:
            if imag_eig > 0 and len(F_plus) < 2:
                F_plus.append(v_norm)
            elif len(F_minus) < 2:
                F_minus.append(v_norm)
    
    # Ensure exactly two modes per direction
    if len(F_plus) < 2:
        remaining = [m for m in modes if m[0] not in [v for v in F_plus + F_minus]]
        remaining.sort(key=lambda x: x[2], reverse=True)
        for v_j, _, _, _ in remaining:
            if len(F_plus) < 2:
                norm = np.linalg.norm(v_j) or 1.0
                F_plus.append(v_j / norm)
    
    if len(F_minus) < 2:
        remaining = [m for m in modes if m[0] not in [v for v in F_plus + F_minus]]
        remaining.sort(key=lambda x: x[2])
        for v_j, _, _, _ in remaining:
            if len(F_minus) < 2:
                norm = np.linalg.norm(v_j) or 1.0
                F_minus.append(v_j / norm)
    
    F_plus = np.column_stack(F_plus[:2]) if F_plus else np.zeros((4, 2))
    F_minus = np.column_stack(F_minus[:2]) if F_minus else np.zeros((4, 2))
    
    print(f"step3_general output:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    return F_plus, F_minus

def step3_degenerate(A, epsilon, kx, ky, k0):
    """
    Exception case: Degenerate eigenvalues.
    Use analytical modes for isotropic normal incidence or eigenvalue sorting with orthogonalization.
    """
    print("--- step3_degenerate: Processing degenerate case ---")
    # Check for isotropic normal incidence
    if abs(kx) < 1e-10 and abs(ky) < 1e-10 and np.allclose(epsilon, np.diag([epsilon[0,0], epsilon[0,0], epsilon[0,0]]), atol=1e-6):
        print("Isotropic normal incidence, using analytical modes.")
        n = np.sqrt(epsilon[0,0])
        kz = n * k0
        # TE mode: Ey = 1, Hx = 0
        v_te_plus = np.array([0, 1, 0, n]) / np.sqrt(1 + n**2)
        v_te_minus = np.array([0, 1, 0, -n]) / np.sqrt(1 + n**2)
        # TM mode: Ex = 1, Hy = 0
        v_tm_plus = np.array([1, 0, -n, 0]) / np.sqrt(1 + n**2)
        v_tm_minus = np.array([1, 0, n, 0]) / np.sqrt(1 + n**2)
        print(f"Analytical modes:\nv_te_plus: {v_te_plus}\nv_te_minus: {v_te_minus}\nv_tm_plus: {v_tm_plus}\nv_tm_minus: {v_tm_minus}")
        F_plus = np.column_stack([v_te_plus, v_tm_plus])
        F_minus = np.column_stack([v_te_minus, v_tm_minus])
        return F_plus, F_minus
    
    # General degenerate case: Sort by eigenvalue and orthogonalize
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"Eigenvalues: {eigenvalues}")
    F_plus = []
    F_minus = []
    tolerance = 1e-6
    
    indices = np.argsort(np.imag(eigenvalues))
    forward_indices = indices[-2:]
    backward_indices = indices[:2]
    
    for i in forward_indices:
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        norm = np.sqrt(abs(Sz)) if abs(Sz) > tolerance else np.linalg.norm(v_j)
        print(f"Forward eigenvector {i+1}: {v_j}, S_z: {Sz}, norm: {norm}")
        F_plus.append(v_j / norm)
    
    for i in backward_indices:
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        norm = np.sqrt(abs(Sz)) if abs(Sz) > tolerance else np.linalg.norm(v_j)
        print(f"Backward eigenvector {i+1}: {v_j}, S_z: {Sz}, norm: {norm}")
        F_minus.append(v_j / norm)
    
    # Orthogonalize if degenerate
    def orthogonalize_vectors(vectors):
        if len(vectors) < 2:
            return vectors
        v1, v2 = vectors
        v2 = v2 - np.dot(v2, v1) / np.dot(v1, v1) * v1
        norm = np.linalg.norm(v2)
        if norm > tolerance:
            v2 = v2 / norm
        else:
            v2 = np.zeros_like(v2)
        return [v1, v2]
    
    F_plus = orthogonalize_vectors(F_plus)
    F_minus = orthogonalize_vectors(F_minus)
    
    F_plus = np.column_stack(F_plus)
    F_minus = np.column_stack(F_minus)
    
    # Ensure 4x2
    if F_plus.shape[1] < 2:
        F_plus = np.pad(F_plus, ((0, 0), (0, 2 - F_plus.shape[1])), mode='constant')
    if F_minus.shape[1] < 2:
        F_minus = np.pad(F_minus, ((0, 0), (0, 2 - F_minus.shape[1])), mode='constant')
    
    print(f"step3_degenerate output:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    return F_plus, F_minus

In [7]:

# --- Tests ---


def test_step3_normal_incidence_degenerate():
    """
    Test for normal incidence in an isotropic medium.
    Expects two forward and two backward modes, handled by step3_degenerate.
    Includes debugging output for critical steps.
    """
    print("\n--- Starting test_step3_normal_incidence_degenerate ---")
    
    # Input parameters
    k0 = 1.0
    epsilon_iso = np.diag([2.25, 2.25, 2.25])
    kx = 0.0
    ky = 0.0
    print(f"Input parameters:\nk0: {k0}\nepsilon:\n{epsilon_iso}\nkx: {kx}\nky: {ky}")
    
    # Step 2: Build matrix A
    print("\nComputing Berreman matrix A...")
    A = step2_build_matrix_A(epsilon_iso, kx, ky, k0)
    print(f"Berreman matrix A:\n{A}")
    
    # Verify A is 4x4 and Hermitian (for debugging)
    assert A.shape == (4, 4), f"Expected A to be 4x4, got {A.shape}"
    print(f"Shape of A: {A.shape}")
    
    # Step 3: Compute modes
    print("\nCalling step3_modes...")
    F_plus, F_minus = step3_modes(A, epsilon_iso, kx, ky, k0)
    
    # Debugging: Eigenvalues and eigenvectors from step3_modes
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"\nEigenvalues of A:\n{eigenvalues}")
    print(f"Eigenvectors of A:\n{eigenvectors}")
    
    # Compute Poynting vector S_z for each eigenvector
    print("\nComputing Poynting vector S_z for each eigenvector...")
    for i in range(4):
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        print(f"Eigenvector {i+1}: {v_j}\nS_z: {Sz}")
    
    # Output modal bases
    print(f"\nComputed modal bases:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    print(f"F_plus shape: {F_plus.shape}\nF_minus shape: {F_minus.shape}")
    
    # Assertions
    assert F_plus.shape == (4, 2), f"Expected F_plus shape (4, 2), got {F_plus.shape}"
    assert F_minus.shape == (4, 2), f"Expected F_minus shape (4, 2), got {F_minus.shape}"
    
    # Verify normalization
    print("\nVerifying normalization of modal bases...")
    for i, vec in enumerate(F_plus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_plus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6)
    
    for i, vec in enumerate(F_minus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_minus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6)
    
    print("\n✅ Test for normal incidence degenerate passed.")


def test_step3_oblique_incidence_isotropic():
    """
    Test for oblique incidence in an isotropic medium (non-degenerate case).
    Expects two forward and two backward modes, handled by step3_general.
    Includes debugging output for critical steps.
    """
    print("\n--- Starting test_step3_oblique_incidence_isotropic ---")
    
    # Input parameters
    wavelength = 550e-9
    k0 = 2 * math.pi / wavelength
    layers = [{'n_in': 1.5}]  # n_in = 1.5
    theta_rad = math.radians(45)
    phi_rad = math.radians(0)
    print(f"Input parameters:\nwavelength: {wavelength}\nk0: {k0}\nlayers: {layers}\ntheta: {math.degrees(theta_rad)} deg\nphi: {math.degrees(phi_rad)} deg")
    
    # Step 1: Compute constants
    print("\nComputing constants from step1_definitions...")
    constants = step1_definitions(wavelength, layers, theta_rad, phi_rad)
    kx = constants['kx']
    ky = constants['ky']
    print(f"Computed constants:\nkx: {kx}\nky: {ky}")
    
    # Define isotropic permittivity
    epsilon_iso = np.diag([2.25, 2.25, 2.25])  # n = sqrt(2.25) = 1.5
    print(f"Permittivity tensor:\n{epsilon_iso}")
    
    # Step 2: Build matrix A
    print("\nComputing Berreman matrix A...")
    A = step2_build_matrix_A(epsilon_iso, kx, ky, k0)
    print(f"Berreman matrix A:\n{A}")
    
    # Verify A is 4x4
    assert A.shape == (4, 4), f"Expected A to be 4x4, got {A.shape}"
    print(f"Shape of A: {A.shape}")
    
    # Step 3: Compute modes
    print("\nCalling step3_modes...")
    F_plus, F_minus = step3_modes(A, epsilon_iso, kx, ky, k0)
    
    # Debugging: Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"\nEigenvalues of A:\n{eigenvalues}")
    print(f"Eigenvectors of A:\n{eigenvectors}")
    
    # Compute Poynting vector S_z for each eigenvector
    print("\nComputing Poynting vector S_z for each eigenvector...")
    for i in range(4):
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        print(f"Eigenvector {i+1}: {v_j}\nS_z: {Sz}")
    
    # Output modal bases
    print(f"\nComputed modal bases:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    print(f"F_plus shape: {F_plus.shape}\nF_minus shape: {F_minus.shape}")
    
    # Assertions for shape
    assert F_plus.shape == (4, 2), f"Expected F_plus shape (4, 2), got {F_plus.shape}"
    assert F_minus.shape == (4, 2), f"Expected F_minus shape (4, 2), got {F_minus.shape}"
    
    # Verify normalization
    print("\nVerifying normalization of modal bases...")
    for i, vec in enumerate(F_plus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_plus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_plus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    for i, vec in enumerate(F_minus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_minus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_minus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    print("\n✅ Test for oblique incidence isotropic passed.")
import pytest
import numpy as np
import cmath
import math


def test_step3_anisotropic_oblique_parallel():
    """
    Test for oblique incidence (30°) parallel to the slow axis (z-axis) in an anisotropic medium.
    Expects two forward and two backward modes, handled by step3_general.
    Includes debugging output for critical steps.
    """
    print("\n--- Starting test_step3_anisotropic_oblique_parallel ---")
    
    # Input parameters
    wavelength = 550e-9
    k0 = 2 * math.pi / wavelength
    layers = [{'n_in': 1.5}]  # Incident medium n = 1.5
    theta_rad = math.radians(30)
    phi_rad = math.radians(90)  # Parallel to z-axis (ky != 0, kx = 0)
    print(f"Input parameters:\nwavelength: {wavelength}\nk0: {k0}\nlayers: {layers}\ntheta: {math.degrees(theta_rad)} deg\nphi: {math.degrees(phi_rad)} deg")
    
    # Step 1: Compute constants
    print("\nComputing constants from step1_definitions...")
    constants = step1_definitions(wavelength, layers, theta_rad, phi_rad)
    kx = constants['kx']
    ky = constants['ky']
    print(f"Computed constants:\nkx: {kx}\nky: {ky}")
    
    # Define anisotropic permittivity (biaxial)
    epsilon_aniso = np.diag([2.5, 2.6, 2.7])  # Slow axis: z (epsilon_z = 2.7)
    print(f"Permittivity tensor:\n{epsilon_aniso}")
    
    # Step 2: Build matrix A
    print("\nComputing Berreman matrix A...")
    A = step2_build_matrix_A(epsilon_aniso, kx, ky, k0)
    print(f"Berreman matrix A:\n{A}")
    
    # Verify A is 4x4
    assert A.shape == (4, 4), f"Expected A to be 4x4, got {A.shape}"
    print(f"Shape of A: {A.shape}")
    
    # Step 3: Compute modes
    print("\nCalling step3_modes...")
    F_plus, F_minus = step3_modes(A, epsilon_aniso, kx, ky, k0)
    
    # Debugging: Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"\nEigenvalues of A:\n{eigenvalues}")
    print(f"Eigenvectors of A:\n{eigenvectors}")
    
    # Compute Poynting vector S_z for each eigenvector
    print("\nComputing Poynting vector S_z for each eigenvector...")
    for i in range(4):
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        print(f"Eigenvector {i+1}: {v_j}\nS_z: {Sz}")
    
    # Output modal bases
    print(f"\nComputed modal bases:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    print(f"F_plus shape: {F_plus.shape}\nF_minus shape: {F_minus.shape}")
    
    # Assertions for shape
    assert F_plus.shape == (4, 2), f"Expected F_plus shape (4, 2), got {F_plus.shape}"
    assert F_minus.shape == (4, 2), f"Expected F_minus shape (4, 2), got {F_minus.shape}"
    
    # Verify normalization
    print("\nVerifying normalization of modal bases...")
    for i, vec in enumerate(F_plus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_plus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_plus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    for i, vec in enumerate(F_minus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_minus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_minus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    print("\n✅ Test for anisotropic oblique parallel passed.")


def test_step3_anisotropic_oblique_perpendicular():
    """
    Test for oblique incidence (30°) perpendicular to the slow axis (z-axis) in an anisotropic medium.
    Expects two forward and two backward modes, handled by step3_general.
    Includes debugging output for critical steps.
    """
    print("\n--- Starting test_step3_anisotropic_oblique_perpendicular ---")
    
    # Input parameters
    wavelength = 550e-9
    k0 = 2 * math.pi / wavelength
    layers = [{'n_in': 1.5}]  # Incident medium n = 1.5
    theta_rad = math.radians(30)
    phi_rad = math.radians(0)  # Perpendicular to z-axis (kx != 0, ky = 0)
    print(f"Input parameters:\nwavelength: {wavelength}\nk0: {k0}\nlayers: {layers}\ntheta: {math.degrees(theta_rad)} deg\nphi: {math.degrees(phi_rad)} deg")
    
    # Step 1: Compute constants
    print("\nComputing constants from step1_definitions...")
    constants = step1_definitions(wavelength, layers, theta_rad, phi_rad)
    kx = constants['kx']
    ky = constants['ky']
    print(f"Computed constants:\nkx: {kx}\nky: {ky}")
    
    # Define anisotropic permittivity (biaxial)
    epsilon_aniso = np.diag([2.5, 2.6, 2.7])  # Slow axis: z (epsilon_z = 2.7)
    print(f"Permittivity tensor:\n{epsilon_aniso}")
    
    # Step 2: Build matrix A
    print("\nComputing Berreman matrix A...")
    A = step2_build_matrix_A(epsilon_aniso, kx, ky, k0)
    print(f"Berreman matrix A:\n{A}")
    
    # Verify A is 4x4
    assert A.shape == (4, 4), f"Expected A to be 4x4, got {A.shape}"
    print(f"Shape of A: {A.shape}")
    
    # Step 3: Compute modes
    print("\nCalling step3_modes...")
    F_plus, F_minus = step3_modes(A, epsilon_aniso, kx, ky, k0)
    
    # Debugging: Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"\nEigenvalues of A:\n{eigenvalues}")
    print(f"Eigenvectors of A:\n{eigenvectors}")
    
    # Compute Poynting vector S_z for each eigenvector
    print("\nComputing Poynting vector S_z for each eigenvector...")
    for i in range(4):
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        print(f"Eigenvector {i+1}: {v_j}\nS_z: {Sz}")
    
    # Output modal bases
    print(f"\nComputed modal bases:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    print(f"F_plus shape: {F_plus.shape}\nF_minus shape: {F_minus.shape}")
    
    # Assertions for shape
    assert F_plus.shape == (4, 2), f"Expected F_plus shape (4, 2), got {F_plus.shape}"
    assert F_minus.shape == (4, 2), f"Expected F_minus shape (4, 2), got {F_minus.shape}"
    
    # Verify normalization
    print("\nVerifying normalization of modal bases...")
    for i, vec in enumerate(F_plus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_plus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_plus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    for i, vec in enumerate(F_minus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_minus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_minus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    print("\n✅ Test for anisotropic oblique perpendicular passed.")


def test_step3_anisotropic_oblique_not_aligned():
    """
    Test for oblique incidence (30°) not aligned to any optical axis in an anisotropic medium.
    Expects two forward and two backward modes, handled by step3_general.
    Includes debugging output for critical steps.
    """
    print("\n--- Starting test_step3_anisotropic_oblique_not_aligned ---")
    
    # Input parameters
    wavelength = 550e-9
    k0 = 2 * math.pi / wavelength
    layers = [{'n_in': 1.5}]  # Incident medium n = 1.5
    theta_rad = math.radians(30)
    phi_rad = math.radians(45)  # Not aligned (kx != 0, ky != 0)
    print(f"Input parameters:\nwavelength: {wavelength}\nk0: {k0}\nlayers: {layers}\ntheta: {math.degrees(theta_rad)} deg\nphi: {math.degrees(phi_rad)} deg")
    
    # Step 1: Compute constants
    print("\nComputing constants from step1_definitions...")
    constants = step1_definitions(wavelength, layers, theta_rad, phi_rad)
    kx = constants['kx']
    ky = constants['ky']
    print(f"Computed constants:\nkx: {kx}\nky: {ky}")
    
    # Define anisotropic permittivity (biaxial)
    epsilon_aniso = np.diag([2.5, 2.6, 2.7])  # Slow axis: z (epsilon_z = 2.7)
    print(f"Permittivity tensor:\n{epsilon_aniso}")
    
    # Step 2: Build matrix A
    print("\nComputing Berreman matrix A...")
    A = step2_build_matrix_A(epsilon_aniso, kx, ky, k0)
    print(f"Berreman matrix A:\n{A}")
    
    # Verify A is 4x4
    assert A.shape == (4, 4), f"Expected A to be 4x4, got {A.shape}"
    print(f"Shape of A: {A.shape}")
    
    # Step 3: Compute modes
    print("\nCalling step3_modes...")
    F_plus, F_minus = step3_modes(A, epsilon_aniso, kx, ky, k0)
    
    # Debugging: Eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(A)
    print(f"\nEigenvalues of A:\n{eigenvalues}")
    print(f"Eigenvectors of A:\n{eigenvectors}")
    
    # Compute Poynting vector S_z for each eigenvector
    print("\nComputing Poynting vector S_z for each eigenvector...")
    for i in range(4):
        v_j = eigenvectors[:, i]
        Ex, Ey, Hx, Hy = v_j
        Sz = 0.5 * np.real(Ex * np.conj(Hy) - Ey * np.conj(Hx))
        print(f"Eigenvector {i+1}: {v_j}\nS_z: {Sz}")
    
    # Output modal bases
    print(f"\nComputed modal bases:\nF_plus:\n{F_plus}\nF_minus:\n{F_minus}")
    print(f"F_plus shape: {F_plus.shape}\nF_minus shape: {F_minus.shape}")
    
    # Assertions for shape
    assert F_plus.shape == (4, 2), f"Expected F_plus shape (4, 2), got {F_plus.shape}"
    assert F_minus.shape == (4, 2), f"Expected F_minus shape (4, 2), got {F_minus.shape}"
    
    # Verify normalization
    print("\nVerifying normalization of modal bases...")
    for i, vec in enumerate(F_plus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_plus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_plus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    for i, vec in enumerate(F_minus.T):
        Sz = 0.5 * np.real(vec[0] * np.conj(vec[3]) - vec[1] * np.conj(vec[2]))
        norm = np.linalg.norm(vec)
        print(f"F_minus vector {i+1}: S_z = {Sz}, norm = {norm}")
        assert abs(Sz) == pytest.approx(1.0, abs=1e-6) or norm == pytest.approx(1.0, abs=1e-6), f"F_minus vector {i+1} normalization failed: S_z = {Sz}, norm = {norm}"
    
    print("\n✅ Test for anisotropic oblique not aligned passed.")

In [8]:
if __name__ == "__main__":
    # Puedes cambiar este número para ejecutar los tests de otro paso
    run_tests_for_step(3)

--- Ejecutando todos los tests para el Paso 3 ---

Ejecutando: test_step3_normal_incidence_degenerate...

--- Starting test_step3_normal_incidence_degenerate ---
Input parameters:
k0: 1.0
epsilon:
[[2.25 0.   0.  ]
 [0.   2.25 0.  ]
 [0.   0.   2.25]]
kx: 0.0
ky: 0.0

Computing Berreman matrix A...
Using the CORRECTED version of step2_build_matrix_A.
Berreman matrix A:
[[0.+0.j         0.+0.j         0.+0.j         0.+1.j        ]
 [0.+0.j         0.+0.j         0.-1.j         0.+0.j        ]
 [0.+0.j         0.+0.j         0.+0.44444444j 0.+0.44444444j]
 [0.+0.j         0.+0.j         0.+0.j         0.+0.j        ]]
Shape of A: (4, 4)

Calling step3_modes...
--- Step 3: Mode Calculation ---
Eigenvalues: [0.+0.j         0.+0.j         0.+0.44444444j 0.+0.j        ]
Degeneracy detected, routing to step3_degenerate.
--- step3_degenerate: Processing degenerate case ---
Isotropic normal incidence, using analytical modes.
Analytical modes:
v_te_plus: [0.         0.5547002  0.         0.8320

## step 4

In [18]:
def step4_propagator_and_s_matrices(A, d, F_j_plus, F_j_minus, F_k_plus, F_k_minus, kx=0, ky=0, n_j=1.0, n_k=1.0):
    """
    Calculate the propagation matrix T and scattering matrix S for a layer interface.
    
    Parameters:
    - A: 4x4 complex numpy array, Berreman system matrix.
    - d: Layer thickness (meters).
    - F_j_plus, F_j_minus: 4x2 numpy arrays, forward/backward modal bases for layer j.
    - F_k_plus, F_k_minus: 4x2 numpy arrays, forward/backward modal bases for layer k.
    - kx, ky: Wavevector components (1/m).
    - n_j, n_k: Refractive indices of layers j and k (default 1.0).
    
    Returns:
    - T: 4x4 complex numpy array, propagation matrix.
    - S_matrix: 4x4 complex numpy array, scattering matrix.
    """
    print("\n--- Paso 4: Calculando T y S ---")
    if A.shape != (4, 4):
        raise ValueError(f"Matrix A must be 4x4, got {A.shape}")
    if not np.isscalar(d) or d < 0:
        raise ValueError(f"Layer thickness d must be a non-negative scalar, got {d}")
    if not np.isscalar(n_j) or n_j <= 0 or not np.isscalar(n_k) or n_k <= 0:
        raise ValueError(f"Refractive indices n_j and n_k must be positive scalars, got {n_j}, {n_k}")
    for F, name in [(F_j_plus, "F_j_plus"), (F_j_minus, "F_j_minus"), 
                    (F_k_plus, "F_k_plus"), (F_k_minus, "F_k_minus")]:
        if F.shape != (4, 2):
            raise ValueError(f"{name} must be 4x2, got {F.shape}")
    
    k0 = np.abs(A[0, 3])
    is_normal_incidence = np.abs(kx) < 1e-10 and np.abs(ky) < 1e-10
    kz_j = np.sqrt(n_j**2 * k0**2 - kx**2 - ky**2 + 0j)
    phase_term = kz_j * d
    print(f"Phase term (kz_j * d): {phase_term}")
    
    # Compute propagation matrix T
    try:
        if is_normal_incidence:
            phase = np.exp(1j * kz_j * d)
            T = np.diag([phase, phase, np.conj(phase), np.conj(phase)])
        else:
            F = np.hstack((F_j_plus, F_j_minus))
            Q, R = qr(F)
            F = Q
            if np.abs(np.linalg.det(F)) < 1e-10:
                raise ValueError(f"F matrix is singular or nearly singular (det = {np.linalg.det(F)})")
            phase = np.exp(1j * kz_j * d)
            exp_lambda_d = np.diag([phase, phase, np.conj(phase), np.conj(phase)])
            F_inv = np.linalg.inv(F)
            T = F @ exp_lambda_d @ F_inv
        if np.any(np.isnan(T)) or np.any(np.isinf(T)):
            raise ValueError("Propagation matrix T contains NaN or Inf values")
    except Exception as e:
        raise ValueError(f"Failed to compute propagation matrix T: {str(e)}")
    print(f"Matriz de propagación T:\n{T}")
    
    # Compute scattering matrix S using Fresnel coefficients
    try:
        theta_j = np.arcsin(np.sqrt(kx**2 + ky**2) / (n_j * k0)) if n_j * k0 != 0 else 0.0
        theta_k = np.arcsin(np.sqrt(kx**2 + ky**2) / (n_k * k0)) if n_k * k0 != 0 else 0.0
        cos_theta_j = np.cos(theta_j)
        cos_theta_k = np.cos(theta_k)
        
        # Fresnel coefficients
        r_s = (n_j * cos_theta_j - n_k * cos_theta_k) / (n_j * cos_theta_j + n_k * cos_theta_k + 1e-10)
        r_p = (n_k * cos_theta_j - n_j * cos_theta_k) / (n_k * cos_theta_j + n_j * cos_theta_k + 1e-10)
        t_s = 2 * n_j * cos_theta_j / (n_j * cos_theta_j + n_k * cos_theta_k + 1e-10)
        t_p = 2 * n_j * cos_theta_j / (n_k * cos_theta_j + n_j * cos_theta_k + 1e-10)
        
        # Construct 2x2 reflection and transmission matrices
        r_jk = np.array([[r_s, 0], [0, r_p]])
        t_jk = np.array([[t_s, 0], [0, t_p]])
        r_kj = np.array([[-r_s, 0], [0, -r_p]])
        t_kj = np.array([[t_s * (n_k / n_j), 0], [0, t_p * (n_k / n_j)]])
        
        # Transform to modal basis
        F_j = np.hstack((F_j_plus, F_j_minus))
        F_k = np.hstack((F_k_plus, F_k_minus))
        Q_j, _ = qr(F_j)
        Q_k, _ = qr(F_k)
        F_j = Q_j
        F_k = Q_k
        F_j_inv = np.linalg.inv(F_j)
        F_k_inv = np.linalg.inv(F_k)
        
        S_11 = F_j @ r_jk @ F_j_inv[:2, :2]
        S_12 = F_j @ t_kj @ F_k_inv[:2, :2]
        S_21 = F_k @ t_jk @ F_j_inv[2:, :2]
        S_22 = F_k @ r_kj @ F_k_inv[2:, :2]
        
        S_matrix = np.vstack([np.hstack([S_11, S_12]), np.hstack([S_21, S_22])])
        
        # Check unitarity
        S_adjoint = S_matrix.T.conj()
        if not np.allclose(S_adjoint @ S_matrix, np.identity(4), atol=1e-6):
            print(f"Warning: S_matrix is not unitary")
    except np.linalg.LinAlgError:
        raise ValueError("Failed to compute interface matrix S: singular or ill-conditioned")
    print(f"Matriz S de la interfaz:\n{S_matrix}")
    
    return T, S_matrix

In [19]:
import numpy as np
import math
import pytest
from scipy.linalg import expm


def test_step4_S_matrix_high_contrast_interface():
    print("\n--- Test Step 4: S-Matrix Unitarity for High Contrast Interface ---")
    
    wavelength = 800e-9
    theta = math.radians(45)
    phi = math.radians(0)
    n_j = 1.0  # Incident medium (e.g., air)
    n_k = 2.5  # Exit medium (e.g., silicon nitride)
    d = 0.0  # Interface test, no propagation
    
    layers = [{'n_in': n_j}, {'n_in': n_k}]
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    print(f"\n**Constantes calculadas (Paso 1):**")
    print(f"k0: {k0} 1/m")
    print(f"kx: {kx} 1/m")
    print(f"ky: {ky} 1/m")
    
    epsilon_j = np.diag([n_j**2, n_j**2, n_j**2])
    epsilon_k = np.diag([n_k**2, n_k**2, n_k**2])
    A_j = step2_build_matrix_A(epsilon_j, kx, ky, k0)
    A_k = step2_build_matrix_A(epsilon_k, kx, ky, k0)
    
    F_j_plus, F_j_minus = step3_modes(A_j, epsilon_j, kx, ky, k0)
    F_k_plus, F_k_minus = step3_modes(A_k, epsilon_k, kx, ky, k0)
    
    print(f"\n**Matriz de evolución A_j (Paso 2):**\n{A_j}")
    print(f"\n**Bases modales F_j+ y F_j- (Paso 3):**")
    print(f"F_j_plus:\n{F_j_plus}")
    print(f"F_j_minus:\n{F_j_minus}")
    print(f"\n**Matriz de evolución A_k (Paso 2):**\n{A_k}")
    print(f"\n**Bases modales F_k+ y F_k- (Paso 3):**")
    print(f"F_k_plus:\n{F_k_plus}")
    print(f"F_k_minus:\n{F_k_minus}")
    
    T, S_matrix = step4_propagator_and_s_matrices(A_j, d, F_j_plus, F_j_minus, F_k_plus, F_k_minus, kx, ky, n_j, n_k)
    
    print(f"\n**Matrices calculadas (Paso 4):**")
    print(f"Matriz de propagación T:\n{T}")
    print(f"Matriz S de la interfaz:\n{S_matrix}")
    
    # Check unitarity
    S_adjoint = S_matrix.T.conj()
    assert np.allclose(S_adjoint @ S_matrix, np.identity(4), atol=1e-6), f"S_matrix is not unitary: {S_adjoint @ S_matrix}"
    
    # Analytical Fresnel coefficients
    theta_j = np.arcsin(np.sqrt(kx**2 + ky**2) / (n_j * k0)) if n_j * k0 != 0 else 0.0
    theta_k = np.arcsin(np.sqrt(kx**2 + ky**2) / (n_k * k0)) if n_k * k0 != 0 else 0.0
    cos_theta_j = np.cos(theta_j)
    cos_theta_k = np.cos(theta_k)
    r_s = (n_j * cos_theta_j - n_k * cos_theta_k) / (n_j * cos_theta_j + n_k * cos_theta_k)
    r_p = (n_k * cos_theta_j - n_j * cos_theta_k) / (n_k * cos_theta_j + n_j * cos_theta_k)
    t_s = 2 * n_j * cos_theta_j / (n_j * cos_theta_j + n_k * cos_theta_k)
    t_p = 2 * n_j * cos_theta_j / (n_k * cos_theta_j + n_j * cos_theta_k)
    
    # Extract reflection and transmission coefficients
    r_numerical = S_matrix[:2, :2]
    t_numerical = S_matrix[2:, :2]
    r_analytical = np.array([[r_s, 0], [0, r_p]])
    t_analytical = np.array([[t_s, 0], [0, t_p]])
    
    print(f"\n**Analytical vs Numerical Coefficients:**")
    print(f"r_analytical:\n{r_analytical}")
    print(f"r_numerical:\n{r_numerical}")
    print(f"t_analytical:\n{t_analytical}")
    print(f"t_numerical:\n{t_numerical}")
    
    assert np.allclose(np.abs(r_numerical), np.abs(r_analytical), atol=1e-5), f"Reflection coefficients mismatch: got {np.abs(r_numerical)}, expected {np.abs(r_analytical)}"
    assert np.allclose(np.abs(t_numerical), np.abs(t_analytical), atol=1e-5), f"Transmission coefficients mismatch: got {np.abs(t_numerical)}, expected {np.abs(t_analytical)}"
    
    print("✅ High contrast interface test passed.")
    
def test_step4_basic_functionality_corrected():
    print("\n--- Test 4: Funcionalidad Básica Corregida (Thin Film) ---")
    
    wavelength = 800e-9
    theta = math.radians(10)
    phi = math.radians(0)
    d = 100e-9
    n_j = 1.5
    n_k = 1.5
    
    layers = [{'n_in': n_j}, {'n_in': n_k}]
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    print(f"\n**Parámetros de entrada:**")
    print(f"Wavelength: {wavelength} m")
    print(f"Theta: {math.degrees(theta)} degrees")
    print(f"Phi: {math.degrees(phi)} degrees")
    print(f"Layer thickness (d): {d} m")
    
    print(f"\n**Constantes calculadas (Paso 1):**")
    print(f"k0: {k0} 1/m")
    print(f"kx: {kx} 1/m")
    print(f"ky: {ky} 1/m")
    
    epsilon = np.diag([n_j**2, n_j**2, n_j**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    F_plus, F_minus = step3_modes(A, epsilon, kx, ky, k0)
    
    print(f"\n**Matriz de evolución A (Paso 2):**\n{A}")
    print(f"\n**Bases modales F+ y F- (Paso 3):**")
    print(f"F_plus:\n{F_plus}")
    print(f"F_minus:\n{F_minus}")
    
    T, S = step4_propagator_and_s_matrices(A, d, F_plus, F_minus, F_plus, F_minus, kx, ky, n_j, n_k)
    
    print(f"\n**Matrices calculadas (Paso 4):**")
    print(f"Matriz de propagación T:\n{T}")
    print(f"Matriz S de la interfaz:\n{S}")
    
    assert not np.any(np.isnan(T)), "T contains NaN values"
    assert not np.any(np.isinf(T)), "T contains Inf values"
    assert not np.any(np.isnan(S)), "S contains NaN values"
    assert not np.any(np.isinf(S)), "S contains Inf values"
    
    assert T.shape == (4, 4), f"T shape is {T.shape}, expected (4, 4)"
    assert S.shape == (4, 4), f"S shape is {S.shape}, expected (4, 4)"
    
    S_adjoint = S.T.conj()
    assert np.allclose(S_adjoint @ S, np.identity(4), atol=1e-6), "S is not unitary"
    
    print("✅ Test de funcionalidad básica corregido pasado.")
    

def test_step4_T_matrix_numerical_stability():
    print("\n--- Test T-Matrix Numerical Stability (Thin Film) ---")
    
    wavelength = 800e-9
    theta = math.radians(45)
    phi = math.radians(0)
    d = 100e-9
    n_j = 1.5
    n_k = 1.5
    
    layers = [{'n_in': n_j}, {'n_in': n_k}]
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    print(f"\n**Parámetros de entrada:**")
    print(f"Wavelength: {wavelength} m")
    print(f"Theta: {math.degrees(theta)} degrees")
    print(f"Phi: {math.degrees(phi)} degrees")
    print(f"Layer thickness (d): {d} m")
    
    print(f"\n**Constantes calculadas (Paso 1):**")
    print(f"k0: {k0} 1/m")
    print(f"kx: {kx} 1/m")
    print(f"ky: {ky} 1/m")
    
    epsilon = np.diag([n_j**2, n_j**2, n_j**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    F_plus, F_minus = step3_modes(A, epsilon, kx, ky, k0)
    
    print(f"\n**Matriz de evolución A (Paso 2):**\n{A}")
    print(f"\n**Bases modales F+ y F- (Paso 3):**")
    print(f"F_plus:\n{F_plus}")
    print(f"F_minus:\n{F_minus}")
    
    T, _ = step4_propagator_and_s_matrices(A, d, F_plus, F_minus, F_plus, F_minus, kx, ky, n_j, n_k)
    
    print(f"\n**Matriz de propagación T (Paso 4):**\n{T}")
    
    assert not np.any(np.isnan(T)), "T contains NaN values"
    assert not np.any(np.isinf(T)), "T contains Inf values"
    
    T_adjoint = T.T.conj()
    assert np.allclose(T_adjoint @ T, np.identity(4), atol=1e-6), "T is not unitary"
    
    print("✅ Test de estabilidad de la matriz T pasado.")

def test_step4_T_matrix_unitarity():
    print("\n--- Test T-Matrix Unitarity (Normal Incidence) ---")
    
    wavelength = 800e-9
    d = 100e-9
    theta = math.radians(0)
    phi = math.radians(0)
    
    layers = [{'n_in': 1.5}]
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    print(f"\n**Constantes calculadas (Paso 1):**")
    print(f"k0: {k0} 1/m")
    print(f"kx: {kx} 1/m")
    print(f"ky: {ky} 1/m")
    
    epsilon = np.diag([1.5**2, 1.5**2, 1.5**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    print(f"\n**Matriz de evolución A (Paso 2):**\n{A}")
    
    F_plus, F_minus = step3_modes(A, epsilon, kx, ky, k0)
    print(f"\n**Bases modales F+ y F- (Paso 3):**")
    print(f"F_plus:\n{F_plus}")
    print(f"F_minus:\n{F_minus}")
    
    T, _ = step4_propagator_and_s_matrices(A, d, F_plus, F_minus, F_plus, F_minus, kx, ky)
    print(f"\n**Matriz de propagación T (Paso 4):**\n{T}")
    
    T_adjoint = T.T.conj()
    identity = T_adjoint @ T
    assert np.allclose(identity, np.identity(4), atol=1e-6), "T is not unitary"
    print("✅ Test for unitarity of T matrix passed.")


def test_step4_S_matrix_unitarity_interface():
    print("\n--- Test S-Matrix Unitarity Interface ---")
    
    wavelength = 800e-9
    theta = math.radians(45)
    phi = math.radians(0)
    d = 0
    
    layers = [{'n_in': 1.5}]
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    
    print(f"\n**Constantes calculadas (Paso 1):**")
    print(f"k0: {k0} 1/m")
    print(f"kx: {kx} 1/m")
    print(f"ky: {ky} 1/m")
    
    epsilon_1 = np.diag([1.5**2, 1.5**2, 1.5**2])
    A_1 = step2_build_matrix_A(epsilon_1, kx, ky, k0)
    F_plus_1, F_minus_1 = step3_modes(A_1, epsilon_1, kx, ky, k0)
    
    epsilon_2 = np.diag([1.5**2, 1.5**2, 1.5**2])
    A_2 = step2_build_matrix_A(epsilon_2, kx, ky, k0)
    F_plus_2, F_minus_2 = step3_modes(A_2, epsilon_2, kx, ky, k0)
    
    print(f"\n**Matriz de evolución A_1 (Paso 2):**\n{A_1}")
    print(f"\n**Bases modales F+_1 y F-_1 (Paso 3):**")
    print(f"F_plus_1:\n{F_plus_1}")
    print(f"F_minus_1:\n{F_minus_1}")
    print(f"\n**Matriz de evolución A_2 (Paso 2):**\n{A_2}")
    print(f"\n**Bases modales F+_2 y F-_2 (Paso 3):**")
    print(f"F_plus_2:\n{F_plus_2}")
    print(f"F_minus_2:\n{F_minus_2}")
    
    T, S_matrix = step4_propagator_and_s_matrices(A_1, d, F_plus_1, F_minus_1, F_plus_2, F_minus_2, kx, ky)
    
    print(f"\n**Matrices calculadas (Paso 4):**")
    print(f"Matriz de propagación T:\n{T}")
    print(f"Matriz S de la interfaz:\n{S_matrix}")
    
    S_adjoint = S_matrix.T.conj()
    identity = S_adjoint @ S_matrix
    assert np.allclose(identity, np.identity(4), atol=1e-6), "S is not unitary"
    print("✅ Test de unitaridad de la matriz S de interfaz pasado.")
def test_step4_normal_incidence():
    print("\n--- Test Step 4: Normal Incidence ---")
    
    wavelength = 800e-9
    d = 100e-9
    theta = math.radians(0)
    phi = math.radians(0)
    n_j = 1.5
    n_k = 1.5
    
    layers = [{'n_in': n_j}, {'n_in': n_k}]
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    epsilon = np.diag([n_j**2, n_j**2, n_j**2])
    A = step2_build_matrix_A(epsilon, kx, ky, k0)
    F_plus, F_minus = step3_modes(A, epsilon, kx, ky, k0)
    
    T, _ = step4_propagator_and_s_matrices(A, d, F_plus, F_minus, F_plus, F_minus, kx, ky, n_j, n_k)
    
    assert not np.any(np.isnan(T)), "T contains NaN values"
    assert not np.any(np.isinf(T)), "T contains Inf values"
    
    kz = np.sqrt(n_j**2 * k0**2 - kx**2 - ky**2 + 0j)
    phase = np.exp(1j * kz * d)
    expected_T = np.diag([phase, phase, np.conj(phase), np.conj(phase)])
    assert np.allclose(T, expected_T, atol=1e-5), f"T does not match expected diagonal form: got {T}, expected {expected_T}"
    
    print("✅ Normal incidence test passed.")
import numpy as np
import math
import pytest
import matplotlib.pyplot as plt

import numpy as np
import math
import pytest
import matplotlib.pyplot as plt

def test_step4_max_thickness_unitarity():
    print("\n--- Test Step 4: Max Thickness Unitarity ---")
    
    wavelength = 800e-9
    theta = math.radians(30)
    phi = math.radians(0)
    d_start = 10e-9  # 10 nm
    d_max = 10e-6    # 10 µm
    n_values = np.arange(1.0, 4.1, 0.2)  # n from 1.0 to 4.0
    atol = 1e-5
    
    # Logarithmic spacing for 20 points
    d_values = np.logspace(np.log10(d_start), np.log10(d_max), 20)
    
    max_d_unitary = {}
    
    for n in n_values:
        layers = [{'n_in': n}, {'n_in': n}]
        constants = step1_definitions(wavelength, layers, theta, phi)
        k0 = constants['k0']
        kx = constants['kx']
        ky = constants['ky']
        epsilon = np.diag([n**2, n**2, n**2])
        A = step2_build_matrix_A(epsilon, kx, ky, k0)
        F_plus, F_minus = step3_modes(A, epsilon, kx, ky, k0)
        
        max_d = d_start
        for d in d_values:
            T, _ = step4_propagator_and_s_matrices(A, d, F_plus, F_minus, F_plus, F_minus, kx, ky, n_j=n, n_k=n)
            T_adjoint = T.T.conj()
            if np.allclose(T_adjoint @ T, np.identity(4), atol=atol):
                max_d = d
            else:
                break
        
        max_d_unitary[n] = max_d * 1e9  # Convert to nm
    
    # Plot n vs max thickness
    plt.figure(figsize=(6, 4))
    plt.plot(list(max_d_unitary.keys()), list(max_d_unitary.values()), 'o-', label='Max Thickness')
    plt.xlabel('Refractive Index (n)')
    plt.ylabel('Max Thickness (nm)')
    plt.title('Max Thickness for Unitarity vs. n')
    plt.grid(True)
    plt.legend()
    plt.savefig('max_thickness_vs_n.png')
    plt.close()
    
    # Assert unitarity holds for at least 100 nm
    for n, d in max_d_unitary.items():
        assert d >= 100, f"Unitarity lost below 100 nm for n={n}: {d} nm"
    
    print("\nMax thicknesses (nm):", {n: round(d, 2) for n, d in max_d_unitary.items()})
    print("✅ Test passed.")

In [20]:

# --- Bloque de ejecución principal ---
if __name__ == "__main__":
    # Puedes cambiar este número para ejecutar los tests de otro paso
    run_tests_for_step(4)

--- Ejecutando todos los tests para el Paso 4 ---

Ejecutando: test_step4_S_matrix_high_contrast_interface...

--- Test Step 4: S-Matrix Unitarity for High Contrast Interface ---

**Constantes calculadas (Paso 1):**
k0: 7853981.633974483 1/m
kx: 5553603.672697959 1/m
ky: 0.0 1/m
Using the CORRECTED version of step2_build_matrix_A.
Using the CORRECTED version of step2_build_matrix_A.
--- Step 3: Mode Calculation ---
Eigenvalues: [0.00000000e+00+11610685.11884785j 1.16415322e-10 -3756703.48487336j
 0.00000000e+00       +0.j         0.00000000e+00 +5553603.67269796j]
No degeneracy, routing to step3_general.
--- step3_general: Processing non-degenerate case ---
Eigenvector 1: [-0.        +0.j -0.56029442+0.j  0.82829352+0.j -0.        +0.j], S_z: 0.23204412058405488, Im(eigenvalue): 11610685.118847845
Eigenvector 2: [0.        -0.00000000e+00j 0.90211404+0.00000000e+00j
 0.43149769-5.55111512e-17j 0.        -0.00000000e+00j], S_z: -0.19463006356516824, Im(eigenvalue): -3756703.484873363
Ei

C:\Users\esteb\AppData\Local\Temp\ipykernel_9320\1534695072.py:58: RuntimeWarning: invalid value encountered in arcsin
  theta_j = np.arcsin(np.sqrt(kx**2 + ky**2) / (n_j * k0)) if n_j * k0 != 0 else 0.0
C:\Users\esteb\AppData\Local\Temp\ipykernel_9320\1534695072.py:59: RuntimeWarning: invalid value encountered in arcsin
  theta_k = np.arcsin(np.sqrt(kx**2 + ky**2) / (n_k * k0)) if n_k * k0 != 0 else 0.0


## step 5

In [12]:
def step5_compose_and_compute_optics(interface_S_matrices, layer_T_matrices, modal_bases, k0, kx, ky, n_in, n_out, theta_in, theta_out):
    """
    Compose scattering matrices using the Redheffer star product and compute optical coefficients.
    Writes S_total after each layer iteration to a file for debugging.
    
    Parameters:
    - interface_S_matrices: List of 4x4 complex numpy arrays, S-matrices for interfaces.
    - layer_T_matrices: List of 4x4 complex numpy arrays, propagation matrices for layers.
    - modal_bases: List of dicts with 'F_plus' and 'F_minus' (4x2 arrays) for each layer.
    - k0: Wavevector magnitude in vacuum (1/m).
    - kx, ky: Wavevector components (1/m).
    - n_in, n_out: Refractive indices of incident and exit media.
    - theta_in, theta_out: Angles of incidence and exit (radians).
    
    Returns:
    - R: Reflectance.
    - T: Transmittance.
    - A: Absorptance.
    """
    print("\n--- Paso 5: Composición y Coeficientes Ópticos ---")
    
    # Validate inputs
    if not interface_S_matrices or not modal_bases:
        raise ValueError("Lists of S-matrices and modal bases must be non-empty")
    if len(interface_S_matrices) != len(layer_T_matrices) + 1:
        raise ValueError(f"Expected len(interface_S_matrices) = len(layer_T_matrices) + 1, got {len(interface_S_matrices)} and {len(layer_T_matrices)}")
    if len(modal_bases) != len(layer_T_matrices) + 2:
        raise ValueError(f"Expected len(modal_bases) = len(layer_T_matrices) + 2, got {len(modal_bases)} and {len(layer_T_matrices)}")
    for S in interface_S_matrices:
        if S.shape != (4, 4):
            raise ValueError(f"S-matrix must be 4x4, got {S.shape}")
    for T in layer_T_matrices:
        if T.shape != (4, 4):
            raise ValueError(f"T-matrix must be 4x4, got {T.shape}")
    
    # Initialize S_total with the first interface S-matrix
    with open('s_matrix_iterations.txt', 'w') as f:
        f.write("Step 5: S-matrix iterations\n\n")
        S_total = interface_S_matrices[0].copy()
        f.write(f"Layer 0 (Initial S_total):\n{S_total}\n\n")
        print(f"Initial S_total:\n{S_total}")
        
        # Combine S-matrices and T-matrices using the Redheffer star product
        for i in range(1, len(interface_S_matrices)):
            S_a = S_total
            S_b = interface_S_matrices[i]
            T_a = layer_T_matrices[i-1] if i-1 < len(layer_T_matrices) else np.identity(4)
            
            # Apply propagation matrix T_a to S_a
            S_a_11 = S_a[:2, :2]
            S_a_12 = S_a[:2, 2:] @ T_a[2:, 2:]
            S_a_21 = T_a[:2, :2] @ S_a[2:, :2]
            S_a_22 = T_a[:2, 2:] @ S_a[2:, 2:] @ T_a[2:, 2:]
            
            S_b_11 = S_b[:2, :2]
            S_b_12 = S_b[:2, 2:]
            S_b_21 = S_b[2:, :2]
            S_b_22 = S_b[2:, 2:]
            
            I = np.identity(2)
            try:
                D = np.linalg.inv(I - S_b_11 @ S_a_22)
                F = np.linalg.inv(I - S_a_22 @ S_b_11)
            except np.linalg.LinAlgError:
                raise ValueError(f"Singular matrix in Redheffer star product at layer {i}")
            
            S_total_11 = S_a_11 + S_a_12 @ F @ S_b_11 @ S_a_21
            S_total_12 = S_a_12 @ F @ S_b_12
            S_total_21 = S_b_21 @ D @ S_a_21
            S_total_22 = S_b_22 + S_b_21 @ D @ S_a_22 @ S_b_12
            
            S_total = np.vstack([np.hstack([S_total_11, S_total_12]), 
                                 np.hstack([S_total_21, S_total_22])])
            
            f.write(f"Layer {i}:\n{S_total}\n\n")
            print(f"S_total after layer {i}:\n{S_total}")
        
        # Extract reflection and transmission matrices
        r = S_total[:2, :2]
        t = S_total[2:, :2]
        f.write(f"Reflection matrix (r):\n{r}\n\n")
        f.write(f"Transmission matrix (t):\n{t}\n\n")
        print(f"Reflection matrix (r):\n{r}")
        print(f"Transmission matrix (t):\n{t}")
        
        # Compute optical coefficients
        R = np.sum(np.abs(r)**2)
        T_factor = (n_out * np.cos(theta_out)) / (n_in * np.cos(theta_in)) if np.cos(theta_in) != 0 else 1.0
        T = T_factor * np.sum(np.abs(t)**2)
        A = 1 - (R + T)
        
        f.write(f"Reflectance (R): {R}\n")
        f.write(f"Transmittance (T): {T}\n")
        f.write(f"Absorptance (A): {A}\n")
        print(f"Reflectance (R): {R}")
        print(f"Transmittance (T): {T}")
        print(f"Absorptance (A): {A}")
    
    return R, T, A

In [13]:
import numpy as np
import math
from scipy.linalg import qr, expm

def berreman_4x4_solver(layers, wavelength, theta=0.0, phi=0.0):
    """
    Berreman 4x4 solver for multilayer optical systems.
    
    Parameters:
    - layers: List of dicts, each containing 'n_in', 'n_out', 'thickness', or 'permittivity'.
    - wavelength: Wavelength in meters.
    - theta: Angle of incidence (radians).
    - phi: Azimuthal angle (radians).
    
    Returns:
    - R: Reflectance.
    - T: Transmittance.
    - A: Absorptance.
    """
    # Step 1: Definitions and Constants
    constants = step1_definitions(wavelength, layers, theta, phi)
    k0 = constants['k0']
    kx = constants['kx']
    ky = constants['ky']
    n_in = layers[0].get('n_in', 1.0)
    n_out = layers[-1].get('n_out', 1.0)
    theta_in = theta
    theta_out = np.arcsin(np.sin(theta) * n_in / n_out) if n_out != 0 else 0.0
    
    # Lists to store results
    layer_A_matrices = []
    layer_T_matrices = []
    interface_S_matrices = []
    modal_bases = []
    
    # Step 2 and 3: Build A matrices and modal bases for each layer
    for layer in layers:
        epsilon_tensor = layer.get('permittivity')
        if epsilon_tensor is None:
            n = layer.get('n_in') or layer.get('n_out')
            if n is None:
                raise ValueError("Layer must have 'permittivity' or 'n_in'/'n_out'.")
            epsilon_tensor = np.diag([n**2, n**2, n**2])
        
        A = step2_build_matrix_A(epsilon_tensor, kx, ky, k0)
        F_plus, F_minus = step3_modes(A, epsilon_tensor, kx, ky, k0)
        
        layer_A_matrices.append(A)
        modal_bases.append({'F_plus': F_plus, 'F_minus': F_minus})
    
    # Step 4: Propagator and S-Matrices
    for i in range(len(layers) - 1):
        current_layer_data = layers[i]
        next_layer_data = layers[i+1]
        n_j = current_layer_data.get('n_in', 1.0) or current_layer_data.get('n_out', 1.0)
        d = current_layer_data.get('thickness', 0.0) if i > 0 else 0.0
        
        T, S = step4_propagator_and_s_matrices(
            A=layer_A_matrices[i],
            d=d,
            F_j_plus=modal_bases[i]['F_plus'],
            F_j_minus=modal_bases[i]['F_minus'],
            F_k_plus=modal_bases[i+1]['F_plus'],
            F_k_minus=modal_bases[i+1]['F_minus'],
            kx=kx,
            ky=ky,
            n=n_j
        )
        interface_S_matrices.append(S)
        if i > 0:  # Skip T for incident medium
            layer_T_matrices.append(T)
    
    # Step 5: Compose and Compute Optics
    R, T, A = step5_compose_and_compute_optics(
        interface_S_matrices,
        layer_T_matrices,
        modal_bases,
        k0,
        kx,
        ky,
        n_in,
        n_out,
        theta_in,
        theta_out
    )
    
    return R, T, A

In [14]:
def test_step5_energy_conservation():
    print("\n--- Test Step 5: Energy Conservation (Lossless Medium) ---")
    
    wavelength = 800e-9
    theta = math.radians(0)
    phi = math.radians(0)
    n_in = 1.0
    n_out = 1.0
    d = 100e-9
    n = 1.5
    
    layers = [
        {'n_in': n_in},
        {'n_in': n, 'thickness': d},
        {'n_out': n_out}
    ]
    
    R, T, A = berreman_4x4_solver(layers, wavelength, theta, phi)
    
    assert np.isclose(R + T + A, 1.0, atol=1e-5), f"Energy conservation violated: R+T+A={R+T+A}"
    assert R >= 0 and T >= 0, f"Negative R or T: R={R}, T={T}"
    assert abs(A) < 1e-5, f"Non-zero absorptance in lossless medium: A={A}"
    assert os.path.exists('s_matrix_iterations.txt'), "Debug file not created"
    
    with open('s_matrix_iterations.txt', 'r') as f:
        content = f.read()
        assert "Layer 0 (Initial S_total)" in content, "Initial S_total not in debug file"
        assert "Layer 1" in content, "Layer 1 S_total not in debug file"
        assert "Reflectance (R)" in content, "Reflectance not in debug file"
    
    print("✅ Energy conservation test passed.")

In [15]:
# --- Bloque de ejecución principal ---
if __name__ == "__main__":
    # Puedes cambiar este número para ejecutar los tests de otro paso
    run_tests_for_step(5)

--- Ejecutando todos los tests para el Paso 5 ---

Ejecutando: test_step5_energy_conservation...

--- Test Step 5: Energy Conservation (Lossless Medium) ---
Using the CORRECTED version of step2_build_matrix_A.
--- Step 3: Mode Calculation ---
Eigenvalues: [0.      +0.j         0.      +0.j         0.+7853981.63397448j
 0.      +0.j        ]
Degeneracy detected, routing to step3_degenerate.
--- step3_degenerate: Processing degenerate case ---
Isotropic normal incidence, using analytical modes.
Analytical modes:
v_te_plus: [0.         0.70710678 0.         0.70710678]
v_te_minus: [ 0.          0.70710678  0.         -0.70710678]
v_tm_plus: [ 0.70710678  0.         -0.70710678  0.        ]
v_tm_minus: [0.70710678 0.         0.70710678 0.        ]
Using the CORRECTED version of step2_build_matrix_A.
--- Step 3: Mode Calculation ---
Eigenvalues: [0.      +0.j         0.      +0.j         0.+3490658.50398866j
 0.      +0.j        ]
Degeneracy detected, routing to step3_degenerate.
--- step3_

# tester

# solver